In [35]:
import pickle
import numpy as np
import pandas as pd
from scipy.stats import mode
from sklearn.metrics import accuracy_score

In [36]:
window_size = 20
activity_label = 'Walking'
test_segments = np.arange(1,61,2)
n_iters = 100
data = pickle.load(open('./data/mORAL_dataset_for_python_upload_09072020/results/{}/activity_{}_window_size_new_v4_{}.p'.format(activity_label,
                                                                                                                                        activity_label,
                                                                                                                                        window_size),'rb'))
data['embedding'] = data['embedding'].apply(lambda a:a.reshape(1,-1))

In [37]:
def get_boosted_results(df,segment_length):
    df = df.sort_values('timestamp').reset_index(drop=True)
    length_to_sample = int(segment_length*60/window_size)
    if length_to_sample>df.shape[0]-10:
        replace = True
    else:
        replace = False
    y = []
    y_pred_majority = []
    y_pred_maxmean = []
    for i in np.arange(n_iters):
        df_sample = df.sample(length_to_sample,replace=replace)
        embedding = np.concatenate(list(df_sample['embedding']))
        if np.max(np.mean(embedding,axis=0))<.3:
            continue
        y.append(df['label'].values[0])
        y_pred_majority.append(mode(embedding.argmax(axis=1))[0][0])
        y_pred_maxmean.append(np.argmax(np.mean(embedding,axis=0)))
    if len(y)==0:
        return pd.DataFrame([],columns = ['user','majority','maxmean','length'])
    return pd.DataFrame([[df['user'].values[0],accuracy_score(y,y_pred_majority),accuracy_score(y,y_pred_maxmean),segment_length]],
                        columns = ['user','majority','maxmean','length'])
all_df = []
for segment_length in test_segments:
    a = pd.concat([get_boosted_results(df,segment_length) for i,df in data.groupby('user',as_index=False)])
    all_df.append(a)

In [38]:
all_dfs = pd.concat(all_df)

In [39]:
all_dfs.groupby('length').mean()

,majority,maxmean
length,,
1,0.502731,0.606045
3,0.777774,0.782889
5,0.813078,0.829029
7,0.829317,0.830072
9,0.933810,0.934233
11,0.930450,0.931658
13,0.944272,0.942724
15,0.941077,0.941077
17,0.939542,0.939542


In [17]:
np.concatenate(list(data.sample(10)['embedding'])).shape

(10, 23)

In [25]:
all_dfs.user.unique().shape

(21,)